In [ ]:
! pip install transformers bert_score torch pandas --quiet

In [2]:
from transformers import pipeline, set_seed
import torch
from bert_score import score
import pandas as pd
set_seed(42)

In [ ]:
model_name = "gpt2"
text_generator = pipeline(
    "text-generation",
    model=model_name,
)

In [12]:
print(f"Loaded model: {model_name}")

Loaded model: gpt2


In [4]:
prompts = {
    "Direct": "Write a motivational quote about overcoming challenges.",
    "Scenario-based": "Imagine you're encouraging a student who is struggling with self-doubt before an exam. Write a motivational message.",
    "Persona-based": "As a seasoned life coach, write a quote about finding purpose in life.",
    "Keyword-based": "Using the words 'resilience', 'dreams', and 'courage', write an inspiring message.",
    "Conversational": "User: I'm feeling unmotivated and stuck. GPT-2: Here's an inspiring quote for you:"
}

In [5]:
print("=== Designed Prompts ===")
for prompt_type, prompt_text in prompts.items():
    print(f"{prompt_type}: {prompt_text}")

=== Designed Prompts ===
Direct: Write a motivational quote about overcoming challenges.
Scenario-based: Imagine you're encouraging a student who is struggling with self-doubt before an exam. Write a motivational message.
Persona-based: As a seasoned life coach, write a quote about finding purpose in life.
Keyword-based: Using the words 'resilience', 'dreams', and 'courage', write an inspiring message.
Conversational: User: I'm feeling unmotivated and stuck. GPT-2: Here's an inspiring quote for you:


In [6]:
outputs = {}

for prompt_type, prompt_text in prompts.items():
    print(f"\n=== Generating for {prompt_type} ===")
    outputs[prompt_type] = []
    responses = text_generator(
        prompt_text,
        max_length=60,
        num_return_sequences=3,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        truncation=True
    )
    for i, response in enumerate(responses, 1):
        generated_text = response["generated_text"].replace(prompt_text, "").strip()
        outputs[prompt_type].append(generated_text)
        print(f"Output {i}: {generated_text}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



=== Generating for Direct ===


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output 1: 
Output 2: When you're at a place like this, you may be surprised at the level of change you see in the world around you. What you see can be very different from what you'd see in your own environment. And the world around you is changing
Output 3: Try to get in touch with someone who has a similar approach to you.

Take a few moments to listen to a motivational quote you've already heard.

Tell them how you feel about the challenge and the challenge you're facing.

=== Generating for Scenario-based ===


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output 1: If you're going to give the student a job, then you have to give the student a job that is not just a job for them, but a job for them to
Output 2: Write a letter to your teacher. Write a personal note that says, "I am not a student. I am a teacher."

Don't think about your students. Don't
Output 3: If you are not convinced, send a personal letter.

You can use a mobile app to find your nearest therapist.

You can even send your own teacher a

=== Generating for Persona-based ===


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output 1: You might be able to find a way to be happier.

Here are some tips to help you find a way to find your purpose.

1. Don't forget to write a little about yourself.
Output 2: Then come up with a list of tips for your life.

The most important advice you can give your children is to learn about life through the lens of what they are feeling, and what they want to do.
Output 3: The following is an excerpt from the book:

"The key to success is to find a way to make your life a better one. It's easier to go through life without a purpose, when it's

=== Generating for Keyword-based ===


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output 1: "I have never been so happy and inspired as I am now," says Mr. Smith, "but I know that there is more to life than just being good at it.
Output 2: When the world is in need of the most attention, we must ask ourselves: what do we do with it?

The answer is: no.

The world is
Output 3: The message is that the world is changing and that we must change it.

In the past, we've been doing this by doing something that we can't do anymore. We've

=== Generating for Conversational ===
Output 1: "When you've got the power, when you've got the patience, when you've got the courage, when you've got the resolve, when you've got the
Output 2: "If you're not going to get through to the next level, you're going to get stuck."

—Kirk Kibler

The most important thing to
Output 3: "I don't like being a spectator at the top of a stage. If you want to be a spectator, you have to have an identity. I am a spectator


In [7]:
flat_outputs = []
output_labels = []

for prompt_type, output_list in outputs.items():
    for i, output in enumerate(output_list, 1):
        flat_outputs.append(output)
        output_labels.append(f"{prompt_type} {i}")

In [8]:
human_reference = "The greatest glory in living lies not in never falling, but in rising every time we fall."
reference_source = "https://www.brainyquote.com/quotes/nelson_mandela_378967"

print("=== Human-Written Reference ===")
print(f"Quote: {human_reference}")
print(f"Source: {reference_source}")

=== Human-Written Reference ===
Quote: The greatest glory in living lies not in never falling, but in rising every time we fall.
Source: https://www.brainyquote.com/quotes/nelson_mandela_378967


In [9]:
P, R, F1 = score(
    flat_outputs,
    [human_reference] * len(flat_outputs),
    lang="en",
    verbose=False
)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
results = []

for label, f1_score in zip(output_labels, F1.tolist()):
    prompt_type, output_num = label.rsplit(" ", 1)
    results.append({
        "Prompt Type": prompt_type,
        "Output #": output_num,
        "BERTScore F1": round(f1_score, 4)
    })

In [11]:
results_df = pd.DataFrame(results)

print("\n=== BERTScore Results ===")
print(results_df.to_markdown(index=False))


=== BERTScore Results ===
| Prompt Type    |   Output # |   BERTScore F1 |
|:---------------|-----------:|---------------:|
| Direct         |          1 |         0      |
| Direct         |          2 |         0.8416 |
| Direct         |          3 |         0.8332 |
| Scenario-based |          1 |         0.8347 |
| Scenario-based |          2 |         0.8424 |
| Scenario-based |          3 |         0.8266 |
| Persona-based  |          1 |         0.8411 |
| Persona-based  |          2 |         0.8599 |
| Persona-based  |          3 |         0.8536 |
| Keyword-based  |          1 |         0.8547 |
| Keyword-based  |          2 |         0.8416 |
| Keyword-based  |          3 |         0.8472 |
| Conversational |          1 |         0.8024 |
| Conversational |          2 |         0.8238 |
| Conversational |          3 |         0.8461 |
